Install basic dependencies for running NasBench

In [295]:
import numpy as np

# For meta-proxy
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from einops import rearrange
from torch import optim
import torch.nn.functional as F


In [296]:
%cd machop/chop/NASLib

/home/ansonhon/mase_project/machop/chop/NASLib


In [ ]:
%pip install -e . --quiet
%pip install ConfigSpace==0.6 --quiet

In [298]:
# For running cifar10
!source scripts/bash_scripts/download_benchmarks.sh nb201 cifar10

dataset = cifar10
search_space = nb201
cifar10 exists


In [299]:
### Code for generating list for random sampling architecture from NASBench
big_list = []
while len(big_list) < 200:
    small_rand_list = [int(np.random.rand()*5) for _ in range(6)]
    if small_rand_list not in big_list:
        big_list.append(small_rand_list)


In [300]:
# After copying the list for specifying architectures into config file, we ran the following command:
%./ch search --config configs/examples/search_nas.toml

UsageError: Line magic function `%./ch` not found.


#### Evaluate Results

In [ ]:
scores = np.load(r'/home/ansonhon/mase_project/nas_results/scores_test2.npy',allow_pickle=True)
val_accuracies = np.load(r'/home/ansonhon/mase_project/nas_results/val_acc_test2.npy', allow_pickle=True)
val_accuracies /= 100

In [333]:
scores = np.load(r'/home/ansonhon/mase_project/nas_results/scores_test3.npy',allow_pickle=True)
val_accuracies = np.load(r'/home/ansonhon/mase_project/nas_results/val_acc_test3.npy',allow_pickle=True)
val_accuracies /= 100

In [334]:
len(val_accuracies)

500

In [ ]:
proxy = ['epe_nas', 
        'fisher', 
        'grad_norm', 
        'grasp', 
        'jacov', 
        'l2_norm', 
        'nwot', 
        'plain', 
        'snip', 
        'synflow', 
        'zen', 
        'flops', 
        'params']
name = proxy[0]

### Using Neural Network for meta-proxy

In [ ]:
# Prepare data into format for pytorch
data_set = []
for proxy_name in proxy:
    data_set.append(scores.item().get(proxy_name))
data_set = np.array(data_set)

In [ ]:
features = rearrange(data_set, 'a b -> b a')
labels = val_accuracies

In [ ]:
# Create data loader
features = torch.Tensor(features)
labels = torch.Tensor(labels)
dataset = TensorDataset(features, labels)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
# Define Model
class neural_model(nn.Module):
    def __init__(self, input_size):
        super(neural_model, self).__init__()
        self.linear1 = nn.Linear(input_size, input_size)
        self.linear2 = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.sigmoid(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

In [301]:
import torch
import torch.nn as nn

class NeuralModel(nn.Module):
    def __init__(self):
        super(NeuralModel, self).__init__()
        self.linear1 = nn.Linear(13, 64)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(64, 128)
        self.relu = nn.ReLU()
        self.linear3 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))  # 输出层的激活函数使用sigmoid
        return x

# 创建模型实例
model = NeuralModel()
# try this

In [302]:
input_size = len(proxy)
# model = neural_model(input_size)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)


In [ ]:
num_epochs = 1000

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

### Evaluate the data

In [332]:
i = int(np.random.rand()*200)
with torch.no_grad():
    new_data = torch.tensor(features[i], dtype=torch.float32)
    prediction = model(features[i])
    print("averaging proxies gives: ", torch.mean(features[i]))
    print('average accruracy: ', )
    print("Prediction gives: ", prediction)
    print("actual accuracy: ", val_accuracies[i])

averaging proxies gives:  tensor(98.0233)
average accruracy
Prediction gives:  tensor([0.8495])
actual accuracy:  0.8847


/tmp/ipykernel_22320/4139946056.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_data = torch.tensor(features[i], dtype=torch.float32)


In [268]:
torch.mean(features[0])

tensor(76.7071)